In [16]:
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

backend_uri = os.environ.get('MLFLOW_TRACKING_URI')
mlflow.set_tracking_uri(backend_uri)
client = MlflowClient()

<IPython.core.display.Javascript object>

In [17]:
prediction_types = [
    { 'type': 'Regression', 'registered_as': 'linreg'},
    { 'type': 'Classification', 'registered_as': ''}
]

prediction_types = pd.DataFrame(prediction_types)
prediction_types.index += 1

In [18]:
prediction_types

,type,registered_as
1,Regression,linreg
2,Classification,


In [19]:
prediction_types[prediction_types['registered_as'] == '' ]

,type,registered_as
2,Classification,


In [21]:
selected_ptype = 1

In [22]:
name = prediction_types.loc[selected_ptype]['registered_as']
registered_models = pd.DataFrame(columns=['source', 'version', 'stage'])
for mv in client.search_model_versions(f"name='{name}'"):
    mv = dict(mv)
    data = {'source': mv['source'].split('/')[-1], 
            'version': mv['version'], 
            'stage': mv['current_stage']
           }
    registered_models = registered_models.append(data, ignore_index = True)

registered_models.index += 1

In [23]:
registered_models

,source,version,stage
1,mlflow_tf_housing,1,Staging
2,mlflow_tf_housing,2,None
3,mlflow_sk_housing,3,None
4,mlflow_sk_housing,4,None
5,mlflow_custom_housing,5,None
6,mlflow_custom_housing,6,None


In [24]:
cond = (registered_models['stage'] == 'Staging')
print(f"{registered_models[cond]['source'].iloc[0]}:v{registered_models[cond]['version'].iloc[0]}")

mlflow_tf_housing:v1


In [25]:
df = registered_models[~cond]
df

,source,version,stage
2,mlflow_tf_housing,2,None
3,mlflow_sk_housing,3,None
4,mlflow_sk_housing,4,None
5,mlflow_custom_housing,5,None
6,mlflow_custom_housing,6,None


In [26]:
for index, row in registered_models[~cond].iterrows():
    print(f"{row['source']}:v{row['version']}")

mlflow_tf_housing:v2
mlflow_sk_housing:v3
mlflow_sk_housing:v4
mlflow_custom_housing:v5
mlflow_custom_housing:v6


In [27]:
current_version = int(registered_models[cond]['version'].iloc[0])
selected_version = 2

client.transition_model_version_stage(
    name = name,
    version = current_version,
    stage="Archived"
)

client.transition_model_version_stage(
    name = name,
    version = selected_version,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1626929161471, current_stage='Staging', description=None, last_updated_timestamp=1626929571214, name='linreg', run_id='23320a23a02049ca935dbf22ecced8bb', run_link=None, source='s3://mlruns/23320a23a02049ca935dbf22ecced8bb/artifacts/mlflow_tf_housing', status='READY', status_message=None, tags={}, user_id=None, version=2>